<a href="https://colab.research.google.com/github/LoreviceP/desafio_case/blob/main/case_data_engineering_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install aspose-cells

In [ ]:
from urllib import request
import jpype     
import asposecells  
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.pivot.fields import Missing
from openpyxl import workbook, load_workbook
from datetime import datetime

In [ ]:
# File request and download 
file_url = 'http://raw.githubusercontent.com/raizen-analytics/data-engineering-test/master/assets/vendas-combustiveis-m3.xls'
filename = 'vendas_combustiveis_m3.xls'

request.urlretrieve(file_url , filename )

('vendas_combustiveis_m3.xls', <http.client.HTTPMessage at 0x7fecf1e56020>)

In [ ]:
# Convert xls to xlsx file and save a raw_layer
root_path = '/content/'
raw_layer = 'raw'
raw_file = f'{raw_layer}_{filename[:-4]}.xlsx'

jpype.startJVM() 
from asposecells.api import Workbook
workbook = Workbook(f'{root_path}{filename}')
workbook.save(f'{root_path}{raw_file}')
jpype.shutdownJVM()

In [ ]:
# Extract pivot table 'Sales of oil derivative fuels by UF and product'
raw_file_path = f'{root_path}{raw_file}'

workbook = load_workbook(raw_file_path)
worksheet = workbook['Plan1']
pivot_table_name = 'Tabela dinâmica1'

pivot_table = [p for p in worksheet._pivots if p.name == pivot_table_name][0]

# Extract a dict of all cache fields and their respective values
fields_map = {}
for field in pivot_table.cache.cacheFields:
    if field.sharedItems.count > 0:
        fields_map[field.name] = [f.v for f in field.sharedItems._fields]

# Extract all rows from cache records. Each row is initially parsed as a dict
column_names = [field.name for field in pivot_table.cache.cacheFields]
rows = []
for record in pivot_table.cache.records.r:
    # If some field in the record in missing, we replace it by NaN
    record_values = [
        field.v if not isinstance(field, Missing) else np.nan for field in record._fields
    ]

    row_dict = {k: v for k, v in zip(column_names, record_values)}

    # Shared fields are mapped as an Index, so we replace the field index by its value
    for key in fields_map:
        row_dict[key] = fields_map[key][row_dict[key]]

    rows.append(row_dict)


In [ ]:
# Create a pandas dataframe and save a silver layer 
df = pd.DataFrame.from_dict(rows)

silver_layer = 'silver'
silver_file_name = 'sales_of_oil_derivative_fuels_by_UF_and_product'

df.to_parquet(f'{root_path}{silver_layer}_{silver_file_name}.parquet')

In [ ]:
# Some transformations
df.drop(columns=['REGIÃO', 'TOTAL'], inplace=True)

df = df.melt(id_vars=['COMBUSTÍVEL','ANO', 'ESTADO', 'UNIDADE'], var_name = 'MES', value_name='VOLUME')
df['MES'] = df['MES'].replace(['Fev', 'Abr', 'Mai', 'Ago', 'Set', 'Out', 'Dez'], ['Feb', 'Apr', 'May', 'Aug', 'Sep', 'Oct', 'Dec'])

In [ ]:
# Create date column
df = df.astype({'ANO' : int})
df = df.astype({'ANO' : str})
df['CONCAT_DT'] = df['ANO'] + '-' + df['MES'] + '-01'
df['DATE'] = pd.to_datetime(df.CONCAT_DT)
df.drop(columns=['ANO', 'MES', 'CONCAT_DT'], inplace=True)

# Rename columns
df = df[['DATE', 'ESTADO', 'COMBUSTÍVEL', 'UNIDADE', 'VOLUME']]
df.rename(columns={'DATE':'year_month', 'ESTADO' : 'uf', 'COMBUSTÍVEL' : 'product', 'UNIDADE' : 'unity', 'VOLUME' : 'volume' }, inplace=True)

df['created_at'] = datetime.now()

gold_layer = 'gold'
gold_file_name = silver_file_name

df.to_parquet(f'{root_path}{gold_layer}_{gold_file_name}.parquet')

In [ ]:
# Display just to visualization
display(df)

,year_month,uf,product,unity,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,9563.263,2023-05-10 15:25:12.224598
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3065.758,2023-05-10 15:25:12.224598
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,17615.604,2023-05-10 15:25:12.224598
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3259.300,2023-05-10 15:25:12.224598
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,28830.479,2023-05-10 15:25:12.224598
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,NaN,2023-05-10 15:25:12.224598
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,NaN,2023-05-10 15:25:12.224598
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,NaN,2023-05-10 15:25:12.224598
54430,2020-12-01,GOIÁS,GLP (m3),m3,NaN,2023-05-10 15:25:12.224598


In [ ]:
# Extract pivot table 'Sales of diesel by UF and type'
raw_file_path = f'{root_path}{raw_file}'

workbook_1 = load_workbook(raw_file_path)
worksheet_1 = workbook_1['Plan1']
pivot_table_name = 'Tabela dinâmica3'

pivot_table = [p for p in worksheet_1._pivots if p.name == pivot_table_name][0]

# Extract a dict of all cache fields and their respective values
fields_map = {}
for field in pivot_table.cache.cacheFields:
    if field.sharedItems.count > 0:
        fields_map[field.name] = [f.v for f in field.sharedItems._fields]

# Extract all rows from cache records. Each row is initially parsed as a dict
column_names = [field.name for field in pivot_table.cache.cacheFields]
rows_1 = []
for record in pivot_table.cache.records.r:
    # If some field in the record in missing, we replace it by NaN
    record_values = [
        field.v if not isinstance(field, Missing) else np.nan for field in record._fields
    ]

    row_dict = {k: v for k, v in zip(column_names, record_values)}

    # Shared fields are mapped as an Index, so we replace the field index by its value
    for key in fields_map:
        row_dict[key] = fields_map[key][row_dict[key]]

    rows_1.append(row_dict)

In [ ]:
# Create a pandas dataframe and save at silver layer 
df1 = pd.DataFrame.from_dict(rows_1)

silver_layer = 'silver'
silver_file_name = 'sales_of_diesel_by_UF_and_type'

df1.to_parquet(f'{root_path}{silver_layer}_{silver_file_name}.parquet')

In [ ]:
# Some transformations
df1.drop(columns=['REGIÃO', 'TOTAL'], inplace=True)

df1 = df1.melt(id_vars=['COMBUSTÍVEL','ANO', 'ESTADO', 'UNIDADE'], var_name = 'MES', value_name='VOLUME')
df1['MES'] = df1['MES'].replace(['Fev', 'Abr', 'Mai', 'Ago', 'Set', 'Out', 'Dez'], ['Feb', 'Apr', 'May', 'Aug', 'Sep', 'Oct', 'Dec'])

In [ ]:
# Create date column
df1 = df1.astype({'ANO' : int})
df1 = df1.astype({'ANO' : str})
df1['CONCAT_DT'] = df1['ANO'] + '-' + df1['MES'] + '-01'
df1['DATE'] = pd.to_datetime(df1.CONCAT_DT)
df1.drop(columns=['ANO', 'MES', 'CONCAT_DT'], inplace=True)

# Rename columns
df1 = df1[['DATE', 'ESTADO', 'COMBUSTÍVEL', 'UNIDADE', 'VOLUME']]
df1.rename(columns={'DATE':'year_month', 'ESTADO' : 'uf', 'COMBUSTÍVEL' : 'product', 'UNIDADE' : 'unity', 'VOLUME' : 'volume' }, inplace=True)

df1['created_at'] = datetime.now()

gold_layer = 'gold'
gold_file_name = silver_file_name

df1.to_parquet(f'{root_path}{gold_layer}_{gold_file_name}.parquet')

In [ ]:
# Display just to visualization
display(df1)

,year_month,uf,product,unity,volume,created_at
0,2013-01-01,RONDÔNIA,ÓLEO DIESEL S-10 (m3),m3,3517.600,2023-05-10 15:25:50.423868
1,2013-01-01,ACRE,ÓLEO DIESEL S-10 (m3),m3,363.000,2023-05-10 15:25:50.423868
2,2013-01-01,AMAZONAS,ÓLEO DIESEL S-10 (m3),m3,3190.585,2023-05-10 15:25:50.423868
3,2013-01-01,RORAIMA,ÓLEO DIESEL S-10 (m3),m3,795.400,2023-05-10 15:25:50.423868
4,2013-01-01,PARÁ,ÓLEO DIESEL S-10 (m3),m3,30137.800,2023-05-10 15:25:50.423868
...,...,...,...,...,...,...
12955,2020-12-01,RIO GRANDE DO SUL,ÓLEO DIESEL (OUTROS ) (m3),m3,NaN,2023-05-10 15:25:50.423868
12956,2020-12-01,MATO GROSSO DO SUL,ÓLEO DIESEL (OUTROS ) (m3),m3,NaN,2023-05-10 15:25:50.423868
12957,2020-12-01,MATO GROSSO,ÓLEO DIESEL (OUTROS ) (m3),m3,NaN,2023-05-10 15:25:50.423868
12958,2020-12-01,GOIÁS,ÓLEO DIESEL (OUTROS ) (m3),m3,NaN,2023-05-10 15:25:50.423868
